#MISTRAL

In [ ]:
!pip install langchain
!pip install CTransformers
!pip install sentence_transformers
!pip install chromadb
#!pip install pypdf
#!pip install streamlit
#!pip install fastapi
#!pip install jinja2
#!pip install aiofiles
#!pip install PyPDF2
!pip install python-multipart
!pip install faiss-cpu
!pip install google-cloud-aiplatform
!pip install gnews
!pip install torch
!pip install huggingface-hub
!pip -q install accelerate
#!pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers
!pip install unstructured
!pip install ctransformers[cuda]
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.6 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=1c021af30a61252ae9ad536ac4197583963563702850dbff9b860a3ec2cf4706
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.4/502.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.8 MB/s eta 0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.8/115.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.1/517.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=900bfe7802d6dd01b4cfc7e5f5fc6de6e2f224ffdd60f27b2ce8a4b8e09cd753
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6049 sha256=67e1f4821cdf5180ce1c17db918be0bc84cec5d5ab335933a5238f6d700dc779
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5


In [ ]:
from langchain.chains import QAGenerationChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import RetrievalQA
import os
import json
import time
import uvicorn
#import aiofiles
#from PyPDF2 import PdfReader
import csv

In [ ]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from langchain.llms import CTransformers
def load_llm():
    # Load the locally downloaded model here
    llm = CTransformers(
        model = "/content/drive/MyDrive/Colab Notebooks/mistral-7b-instruct-v0.1.Q4_K_S.gguf",
        model_type="mistral",
        max_new_tokens = 2048,
        temperature = 0.3
    )
    return llm

/usr/local/lib/python3.10/dist-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [ ]:
'''def file_processing(file_path):

    # Load data from PDF
    loader = PyPDFLoader(file_path)
    data = loader.load()

    question_gen = ''

    for page in data:
        question_gen += page.page_content

    splitter_ques_gen = RecursiveCharacterTextSplitter(
        chunk_size = 1000,
        chunk_overlap = 100
    )

    chunks_ques_gen = splitter_ques_gen.split_text(question_gen)

    document_ques_gen = [Document(page_content=t) for t in chunks_ques_gen]

    splitter_ans_gen = RecursiveCharacterTextSplitter(
        chunk_size = 300,
        chunk_overlap = 30
    )


    document_answer_gen = splitter_ans_gen.split_documents(
        document_ques_gen
    )

    return document_ques_gen, document_answer_gen'''

In [ ]:
def llm_pipeline(file_path):

    document_ques_gen, document_answer_gen = file_processing(file_path)



    llm_ques_gen_pipeline = load_llm()

    prompt_template = """
    You are an expert at creating questions based on coding materials and documentation.
    Your goal is to prepare a coder or programmer for their exam and coding tests.
    You do this by asking questions about the text below:

    ------------
    {text}
    ------------

    Create questions that will prepare the coders or programmers for their tests.
    Make sure not to lose any important information.

    QUESTIONS:
    """

    PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=["text"])

    refine_template = ("""
    You are an expert at creating practice questions based on coding material and documentation.
    Your goal is to help a coder or programmer prepare for a coding test.
    We have received some practice questions to a certain extent: {existing_answer}.
    We have the option to refine the existing questions or add new ones.
    (only if necessary) with some more context below.
    ------------
    {text}
    ------------

    Given the new context, refine the original questions in English.
    If the context is not helpful, please provide the original questions.
    QUESTIONS:
    """
    )

    REFINE_PROMPT_QUESTIONS = PromptTemplate(
        input_variables=["existing_answer", "text"],
        template=refine_template,
    )

    ques_gen_chain = load_summarize_chain(llm = llm_ques_gen_pipeline,
                                            chain_type = "refine",
                                            verbose = True,
                                            question_prompt=PROMPT_QUESTIONS,
                                            refine_prompt=REFINE_PROMPT_QUESTIONS)

    ques = ques_gen_chain.run(document_ques_gen)

    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

    vector_store = FAISS.from_documents(document_answer_gen, embeddings)

    llm_answer_gen = load_llm()

    ques_list = ques.split("\n")
    filtered_ques_list = [element for element in ques_list if element.endswith('?') or element.endswith('.')]

    answer_generation_chain = RetrievalQA.from_chain_type(llm=llm_answer_gen,
                                                chain_type="stuff",
                                                retriever=vector_store.as_retriever())

    return answer_generation_chain, filtered_ques_list

In [ ]:
def get_csv (file_path):
    answer_generation_chain, ques_list = llm_pipeline(file_path)
    base_folder = 'output/'
    if not os.path.isdir(base_folder):
        os.mkdir(base_folder)
    output_file = base_folder+"QA.csv"
    with open(output_file, "w", newline="", encoding="utf-8") as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["Question", "Answer"])  # Writing the header row

        for question in ques_list:
            print("Question: ", question)
            answer = answer_generation_chain.run(question)
            print("Answer: ", answer)
            print("--------------------------------------------------\n\n")

            # Save answer to CSV file
            csv_writer.writerow([question, answer])
    return output_file

In [ ]:
def file_processing(file_content):
    # Simulating file content processing
    question_gen = file_content

    # Simulating document creation
    splitter_ques_gen = RecursiveCharacterTextSplitter(
        chunk_size=2000,
        chunk_overlap=100
    )
    chunks_ques_gen = splitter_ques_gen.split_text(question_gen)
    document_ques_gen = [Document(page_content=t) for t in chunks_ques_gen]

    # Simulated document splitting
    splitter_ans_gen = RecursiveCharacterTextSplitter(
        chunk_size=2000,
        chunk_overlap=100
    )
    document_answer_gen = splitter_ans_gen.split_documents(document_ques_gen)

    return document_ques_gen, document_answer_gen

# Answer Generation

In [ ]:
from datasets import load_dataset

dataset = load_dataset("ghmfx/natural-questions-short")

/usr/local/lib/python3.10/dist-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# Access the 'train' split of the dataset
train_dataset = dataset["train"]

NameError: ignored

In [ ]:
## https://huggingface.co/datasets/ghmfx/natural-questions-short?row=18

def llm_pipeline_ans(context, question):
    llm_answer_gen=load_llm()
    # Generate prompt for each question and context pair

    prompt_template = """
    Given the following context:

    ------------
    {context}
    ------------

    give short answer for the following question:

    QUESTION:
    {question}
    """

    # Format the prompt for the given context and question
    prompt = prompt_template.format(context=context, question=question)

    # # Use LLM to find the answer
    answer = llm_answer_gen(prompt)

    return answer



In [ ]:

context = train_dataset[0]['contexts']
question = train_dataset[0]['questions']
answer = llm_pipeline_ans(context,question)
print(answer)

NameError: ignored

In [ ]:
# answer generation

import pandas as pd
import os

# Create lists to store data
contexts = []
questions = []
generated_answers = []
actual_answers = []


file_path='/content/drive/MyDrive/'

index = 0

# Check if the file already exists
filename = file_path+f'summary_output/QA_{index}.csv'
while os.path.exists(filename):
  index += 5
  filename = file_path+f'summary_output/QA_{index}.csv'

if index!=0:
  index-=5


for index in range(index,120):
  context = train_dataset[index]['contexts']
  question = train_dataset[index]['questions']
  print("Starting index is: ",index)
  answer = llm_pipeline_ans(context,question)
  contexts.append(context)
  questions.append(question)
  generated_answers.append(answer)
  actual_answers.append(train_dataset[index]['answers'])
  print(f'Answer for {index}: {answer}')

    # Save the final text to a file after every 5 steps
  if index % 5 == 0:
      filename = file_path+f'summary_output/QA_{index}.csv'

      # Create a DataFrame
      df = pd.DataFrame({
          'Context': contexts,
          'Questions': questions,
          'Generated Answers': generated_answers,
          'Actual Answers': actual_answers
      })

      # Save the DataFrame to a CSV file
      df.to_csv(filename,index=False)
      contexts = []
      questions = []
      generated_answers = []
      actual_answers = []

      print(f"Saved file: {filename}.")




print("CSV file created successfully.")

NameError: ignored

In [ ]:
import pandas as pd
import ast
import re

# Specify the file path
file_path = '/content/drive/MyDrive/'
index = 0  # Replace with the actual index value
refined_generated_answers=[]
refined_actual_answers=[]


for fileindex in range(index,111,5):
  filename = f'{file_path}summary_output/QA_{fileindex}.csv'

  # Read the CSV file into a DataFrame
  df = pd.read_csv(filename)

  # Extract 'Generated Answers' and 'Actual Answers' into lists
  generated_answers_list=df['Generated Answers'].tolist()
  actual_answers_list = df['Actual Answers'].to_list()

  for row in range(len(generated_answers_list)):
    generated_answer=str(generated_answers_list[row])
    actual_answer=str(actual_answers_list[row])

    # refine generated answer
    pattern = re.compile(r'ANSWER:(.*)', re.DOTALL)

    # Find all matches
    matches = pattern.findall(generated_answer)

    if len(matches)==0:
      refined_generated_answers.append('NO ANSWER')
    else :
      # store the result
      refined_generated_answers.append(matches[0].strip())



    # refine actual answer

    # Safely evaluate the string as a Python literal
    parsed_dict = ast.literal_eval(actual_answer)

    # Extract 'span_text' from the parsed dictionary
    answer=parsed_dict.get('span_text', None)
    if len(answer)==0:
      refined_actual_answers.append('NO ANSWER')
    else :
      refined_actual_answers.append(answer)


# Display the lists
print("Generated Answers:", refined_generated_answers)
print("Length of the generated answers: ",len(refined_generated_answers))
print("\n--------------------------------------\n")
print("Actual Answers:", refined_actual_answers)
print("Length of the actual answers: ",len(refined_actual_answers))


Generated Answers: ['Stephanie dies in Grey\'s Anatomy during the episode " Ring of Fire " on May 18, 2017.', 'NO ANSWER', 'Cara Black', 'After a president is impeached, the succession goes in the order of vice president , Speaker of the House of Representatives , President pro tempore of the Senate.', 'Shemar Moore played Malcolm Winters on The Young and the Restless.', 'Barry Rothbart voices the dogs in Downward Dog.', 'The Law of Moses, also called the Mosaic Law or Torat Moshe in Hebrew, refers primarily to the Torah or first five books of the Hebrew Bible.', 'Modem', 'The name of Persia changed to Iran in 1935.', 'NO ANSWER', 'The major nations that had won World War I would have supported the Five-Power Naval Treaty of 1922. These nations were the United Kingdom, the United States, Japan, France, and Italy.', "Instagram's own account has the most followers on Instagram in the world.", 'The term used in mathematics to describe how steep a line is is called the slope or gradient of

In [ ]:
!pip install rouge nltk bert-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.1 MB/s eta 0:00:00


In [ ]:
from rouge import Rouge
from nltk.translate.bleu_score import corpus_bleu
from bert_score import score as bert_score

import nltk
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')

from nltk.tokenize import word_tokenize
from nltk.translate.meteor_score import meteor_score
import numpy as np



# ROUGE
rouge = Rouge()
rouge_results = rouge.get_scores(refined_generated_answers,refined_actual_answers, avg=True)
print('\n\n------------------------------\n')
print("ROUGE Scores:", rouge_results)

# BLEU
references = [[summary.split()] for summary in refined_actual_answers]
hypotheses = [summary.split() for summary in refined_generated_answers]
bleu_results = corpus_bleu(references, hypotheses)
print('\n------------------------------\n')
print("BLEU Score:", bleu_results)
print('\n------------------------------\n')

# BERTScore
bert_p, bert_r, bert_f1 = bert_score(refined_generated_answers, refined_actual_answers, lang='en', verbose=True)
print('\n------------------------------\n')
print("BERTScore Precision:", bert_p.mean().item())
print("BERTScore Recall:", bert_r.mean().item())
print("BERTScore F1 Score:", bert_f1.mean().item())
print('\n------------------------------\n')


# METEORScore
meteor_scores=[]

for index in range(len(refined_generated_answers)):
  tokenized_generated_answers = word_tokenize(refined_generated_answers[index].lower())
  tokenized_actual_answers = word_tokenize(refined_actual_answers[index].lower())

  # METEOR
  meteor_scores.append(round(meteor_score([tokenized_generated_answers],tokenized_actual_answers),4))


print('\n------------------------------\n')
print("Average METEOR Score: ",np.mean(meteor_scores))
print('\n------------------------------\n')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!




------------------------------

ROUGE Scores: {'rouge-1': {'r': 0.5136853976659801, 'p': 0.17315893150061784, 'f': 0.22105160481843839}, 'rouge-2': {'r': 0.2630409136624476, 'p': 0.0988414138140228, 'f': 0.11872942389971192}, 'rouge-l': {'r': 0.5068661651185922, 'p': 0.17195786343925945, 'f': 0.21916652883376955}}

------------------------------

BLEU Score: 0.07335536942736223

------------------------------



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 2.22 seconds, 46.45 sentences/sec

------------------------------

BERTScore Precision: 0.8311617374420166
BERTScore Recall: 0.8627310991287231
BERTScore F1 Score: 0.8456557393074036

------------------------------


------------------------------

Average METEOR Score:  0.16813883495145632

------------------------------



# Question Generation

In [ ]:
def llm_pipeline(file_path):


    llm_ques_gen_pipeline = load_llm()

    prompt_template = """
    You are an expert at creating questions based on coding materials and documentation.
    Your goal is to prepare a coder or programmer for their exam and coding tests.
    You do this by asking questions about the text below:

    ------------
    {text}
    ------------

    Create 1 question that will prepare the coders or programmers for their tests.
    Make sure not to lose any important information.

    QUESTION:
    """

    PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=["text"])

    # Format the prompt for the given context and question
    prompt = prompt_template.format(text=context)

    # Use LLM to find the answer
    ques = llm_ques_gen_pipeline(prompt)

    return ques

In [ ]:
# answer generation

import pandas as pd

# Create lists to store data
contexts = []
actual_questions = []
generated_questions = []


file_path='/content/drive/MyDrive/'

index = 0

# Check if the file already exists
filename = file_path+f'summary_output/GeneratedQA_{index}.csv'
while os.path.exists(filename):
  index += 5
  filename = file_path+f'summary_output/GeneratedQA_{index}.csv'

if index!=0:
  index-=5


for row in range(index,100):
  print("Starting at: ",row)
  context = train_dataset[row]['contexts']
  question = train_dataset[row]['questions']
  ques = llm_pipeline(context)


  contexts.append(context)
  generated_questions.append(ques)
  actual_questions.append(question)
  index+=1
  print(f'Question for {row}: {ques}')

  # Save the final text to a file after every 5 steps
  if index % 5 == 0:
      filename = file_path+f'summary_output/GeneratedQA_{index}.csv'

      # Create a DataFrame
      df = pd.DataFrame({
          'Context': contexts,
          'Actual Question': actual_questions,
          'Generated Question': generated_questions
      })

      # Save the DataFrame to a CSV file
      df.to_csv(filename,index=False)
      contexts = []
      actual_questions = []
      generated_questions = []

      print(f"Saved file: {filename}.")




print("CSV file created successfully.")

Starting at:  95


Question for 95: 
What was the yom October War  What is the Yom Kipadaptitude,
Starting at:  96


Question for 96: 
    What was the location of the Battle of Fort Sumter?

    ANSWER: Charleston, South Carolina
    
    What were the coordinates of Fort Sumter during the battle?
    
    ANSWER: 32 ° 45 ′ 8 '' N 79 ° 52 ′ 29 '' W
    
    What was the result of the Battle of Fort Sumter and who captured it?
    
    ANSWER: Confederate victory and the Confederacy captured Fort Sumtermsy captured Fort Sumterrites captured Fort Sumterrit captures captured Fort Sumtermed Confederate captured Fort Sumterms Army under the United States captured Fort Sumtermedallegally captured Fort Sumterritually captured Fort Sumterms army captured Fort Sumtermedy captured Fort Sumterms capture Fort Sumterms troops captured Fort Sumterr captured Fort Sumterrites
Starting at:  97
Question for 97: 
       Which of the following statements best describes the tree line?
       A) It is found at high elevations and in frigid environments.
       B) Beyond it, trees can tolerate the environmental conditions

Question for 99: ------------
    ------------
    ------------
What is it is a)
    1 What is the iPad (iOS: iOS  * iOS  - What is an iPhone The iPad:  What are there was developed  - Which of the iPad in general questions:
What type=========
    --------------
      Question  Ion. Is the iPad  Apple Incred- What is a)
    Apple
    ---------------
1.
    What is it
    1)




    5THISBNDevelopening and  - What is Apple A:
Saved file: /content/drive/MyDrive/summary_output/GeneratedQA_100.csv.
CSV file created successfully.


In [ ]:
def extract_questions(text):
    # Split the text into sentences
    sentences = re.split(r'\n', text)

    # Extract sentences containing a question mark
    question_sentences = [sentence.strip() for sentence in sentences if re.search(r'\?', sentence)]

    # Join the extracted sentences into a single string
    if len(question_sentences)==0:
      return 'NO QUESTION'

    return question_sentences[0]

In [ ]:
import pandas as pd
import ast
import re

# Specify the file path
file_path = '/content/drive/MyDrive/'
index = 0  # Replace with the actual index value
refined_generated_questions=[]
refined_actual_questions=[]


for fileindex in range(index,100,5):
  filename = f'{file_path}summary_output/GeneratedQA_{fileindex}.csv'

  # Read the CSV file into a DataFrame
  df = pd.read_csv(filename)

  # Extract 'Generated Answers' and 'Actual Answers' into lists
  generated_questions_list=df['Generated Question'].tolist()
  actual_questions_list = df['Actual Question'].to_list()

  for row in range(len(generated_questions_list)):
    generated_question=str(generated_questions_list[row])
    actual_question=str(actual_questions_list[row])
    refined_actual_questions.append(actual_question)
    refined_generated_questions.append(extract_questions(generated_question))

# Display the lists
print("Generated Questions:", refined_generated_questions)
print("Length of the generated questions: ",len(refined_generated_questions))
print("\n--------------------------------------\n")
print("Actual Questions:", refined_actual_questions)
print("Length of the actual questions: ",len(refined_actual_questions))


Generated Questions: ["What is the name of the character played by Jerrika Hinton in Grey's Anatomy?", "In this excerpt, what was the main purpose of President Harry S. Truman's speech?", 'What is the name of the event where Cara Black and Leander Paes finished as runners up in 2008?', 'Can you explain the order of succession in the United States presidential line of succession as per the Constitution and Presidential Succession Act of 1947?', 'What is the occupation of Malcolm Winters in The Young and the Restless?', 'What is the name of the production company that created "Downward Dog" and what type of television network did it originally air on?', 'What are the primary sources of information in the Law of Moses, also known as the Mosaic Law or Torat Moshe?', 'How does a modem work?', 'Which word did Reza Shah Pahlavi use to refer to Iran before the Nowruz of 1935, and how has it affected the terms for Iranian nationality and citizenship?', 'NO QUESTION', 'Which treaty was also know

In [ ]:
from rouge import Rouge
from nltk.translate.bleu_score import corpus_bleu
from bert_score import score as bert_score

import nltk
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')

from nltk.tokenize import word_tokenize
from nltk.translate.meteor_score import meteor_score
import numpy as np



# ROUGE
rouge = Rouge()
rouge_results = rouge.get_scores(refined_generated_questions,refined_actual_questions, avg=True)
print('\n\n------------------------------\n')
print("ROUGE Scores:", rouge_results)

# BLEU
references = [[summary.split()] for summary in refined_actual_questions]
hypotheses = [summary.split() for summary in refined_generated_questions]
bleu_results = corpus_bleu(references, hypotheses)
print('\n------------------------------\n')
print("BLEU Score:", bleu_results)
print('\n------------------------------\n')

# BERTScore
bert_p, bert_r, bert_f1 = bert_score(refined_generated_questions, refined_actual_questions, lang='en', verbose=True)
print('\n------------------------------\n')
print("BERTScore Precision:", bert_p.mean().item())
print("BERTScore Recall:", bert_r.mean().item())
print("BERTScore F1 Score:", bert_f1.mean().item())
print('\n------------------------------\n')


# METEORScore
meteor_scores=[]

for index in range(len(refined_generated_questions)):
  tokenized_generated_questions = word_tokenize(refined_generated_questions[index].lower())
  tokenized_actual_questions = word_tokenize(refined_actual_questions[index].lower())

  # METEOR
  meteor_scores.append(round(meteor_score([tokenized_generated_questions],tokenized_actual_questions),4))


print('\n------------------------------\n')
print("Average METEOR Score: ",np.mean(meteor_scores))
print('\n------------------------------\n')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!




------------------------------

ROUGE Scores: {'rouge-1': {'r': 0.24974079739704738, 'p': 0.15836501675623488, 'f': 0.18492264972216887}, 'rouge-2': {'r': 0.07283061267436268, 'p': 0.04417356551052279, 'f': 0.05269692217567897}, 'rouge-l': {'r': 0.22759737369112368, 'p': 0.1459675919040403, 'f': 0.16994619490238372}}

------------------------------

BLEU Score: 0.021027406297706823

------------------------------



config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 4.29 seconds, 22.37 sentences/sec

------------------------------

BERTScore Precision: 0.8549105525016785
BERTScore Recall: 0.8682608604431152
BERTScore F1 Score: 0.8613542914390564

------------------------------


------------------------------

Average METEOR Score:  0.18685833333333335

------------------------------

